## <center> Лабораторная работа №2. Изучение yargy</center>

In [1]:
import yargy
import unittest

In [2]:
from yargy import rule, Parser, and_, or_, not_
from yargy.interpretation import fact
from yargy.relations import gnc_relation
from yargy.predicates import gram, eq, gte, length_eq
from yargy.pipelines import morph_pipeline

### 1. Распознавание ФИО

Стандартные русские имена состоят из имени, отчества и фамилии. Попробуем составить правила для их определения. 

В словаре Opencorpora, который использует pymorphy2, существуют разные метки. Перечислим те, которые могут нам пригодиться в дальнейшем:
    
* `Surn` - для фамилий
* `Name` - для имен
* `Patr` - для отчеств
* `NOUN` - для существительных
* `Geox` - топоним (название какого-либо географического объекта)
* `ADJF` - для прилагательных
* `PREP` - для предлогов
* `NPRO` - для местоимений-существительных
* `Apro` - для местоимений
* `gent` - для родительного падежа
* `Abbr` - для аббревиатур
* `femn` - для женского рода

Создадим объект, в котором будут храниться распарсенные данные:

In [4]:
Name = fact(
    'Name',
    ['first', 'middle', 'last']
)

Где `first` - это имя, `middle` - отчество и `last` - фамилия.

Для согласования по падежу, числу и роду будем использовать механизм согласования.

In [5]:
gnc = gnc_relation()

Итак, подумаем, в каком виде могут встречаться русские имена в текстах. В большинстве случаев это:
* ФИО полностью (Лосева Мария Сергеевна или Мария Сергеевна Лосева)
* Фамилия и имя (Мария Лосева или Лосева Мария) 
* Имя и отчество (Мария Сергеевна)
* Только имя, только фамилия, только отчество (Мария)
* Не встречается ничего (этот случай нужно рассмотреть отдельно)

In [8]:
FIRST = and_(gram('Name'), not_(gram('Abbr'))).interpretation(Name.first).match(gnc)

MIDDLE = and_(gram('Patr'), not_(gram('Abbr'))).interpretation(Name.middle).match(gnc)

LAST = and_(gram('Surn'), not_(gram('Abbr'))).interpretation(Name.last).match(gnc)

Составим правила:

In [9]:
NAME = or_(
    rule(FIRST, MIDDLE, LAST), #Мария Сергеевна Лосева
    rule(LAST, FIRST, MIDDLE.optional()), #Лосева Мария Сергеевна или Лосева Мария
    rule(FIRST, or_(MIDDLE, LAST)), #Мария Сергеевна или Мария Лосева
    rule(FIRST),
    rule(MIDDLE),
    rule(LAST)).interpretation(Name)

In [10]:
NAME_PARSER = Parser(NAME)

#### Тесты для пункта 1 (11 тестов)

In [16]:
class TestPersons(unittest.TestCase):
    
    def setUp(self):
        self.nothing = Name(first = None, middle = None, last = None)
        self.parser = NAME_PARSER
    
    def getRes(self, testing_name):
        find = self.parser.find(testing_name)
        res = self.nothing if find is None else find.fact
        return res
    
    def test_1(self):
        res = self.getRes('Иванов Петр Васильевич')
        self.assertEqual(res.first, 'Петр')
        self.assertEqual(res.middle, 'Васильевич')
        self.assertEqual(res.last, 'Иванов')

    def test_2(self):
        res = self.getRes('шипицын дмитрий вячеславович')
        self.assertEqual(res.first, 'дмитрий')
        self.assertEqual(res.middle, 'вячеславович')
        self.assertEqual(res.last, 'шипицын')

    def test_3(self):
        res = self.getRes('елена владимировна')
        self.assertEqual(res.first, 'елена')
        self.assertEqual(res.middle, 'владимировна')
        self.assertEqual(res.last, None)

    def test_4(self):
        res = self.getRes('басалаева юлия михайловна')
        self.assertEqual(res.first, 'юлия')
        self.assertEqual(res.middle, 'михайловна')
        self.assertEqual(res.last, 'басалаева')

    def test_5(self):
        res = self.getRes('ну я как раз по фамилии есть смотри мам')
        self.assertEqual(res.first, None)
        self.assertEqual(res.middle, None)
        self.assertEqual(res.last, None)

    def test_6(self):
        res = self.getRes('глушенков власти на android')
        self.assertEqual(res.first, None)
        self.assertEqual(res.middle, None)
        self.assertEqual(res.last, 'глушенков')

    def test_7(self):
        res = self.getRes('фамилию сказать что за фамилия терентьева людмила')
        self.assertEqual(res.first, 'людмила')
        self.assertEqual(res.middle, None)
        self.assertEqual(res.last, 'терентьева')

    def test_8(self):
        res = self.getRes('елена владимировна')
        self.assertEqual(res.first, 'елена')
        self.assertEqual(res.middle, 'владимировна')
        self.assertEqual(res.last, None)

    def test_9(self):
        res = self.getRes('анюта')
        self.assertEqual(res.first, 'анюта')
        self.assertEqual(res.middle, None)
        self.assertEqual(res.last, None)

    def test_10(self):
        res = self.getRes('р1 артем витальевич')
        self.assertEqual(res.first, 'артем')
        self.assertEqual(res.middle, 'витальевич')
        self.assertEqual(res.last, None)
        
    def test_11(self):
        res = self.getRes('фитнес веретельников олег викторович')
        self.assertEqual(res.first, 'олег')
        self.assertEqual(res.middle, 'викторович')
        self.assertEqual(res.last, 'веретельников')

In [17]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...........
----------------------------------------------------------------------
Ran 11 tests in 0.083s

OK


### 2. Распознавание адреса

В идеале полный адрес состоит из:
* Города
* Улицы
* Номера дома
* Опционально: номера корпуса или строения
* Номера квартиры (в частных домах его нет)

Суть корпуса и строения приблизительно одинакова, поэтому мы решили их объединить в объекте распарсенных данных под именем `corpus`

In [19]:
Address = fact(
    'Address',
    ['city', 'street', 'building', 'corpus', 'apartment']
)

В составлении правил нам могут понадобиться следующие знаки препинания:

In [20]:
DOT = eq('.')
SLASH = eq('/')
HYPHEN = eq('-')
SEMICOLON = eq(';')

А также числа. Логично, что номера домов/корпусов/строений/квартир - это положительные числа, поэтому:

In [21]:
NUMBER = gte(1)

Будем составлять правила постепенно. Начнем с `города`:

#### Город

Обычно в адресе перед самим названием города могут стоят такие слова как город, гор. или г. Для названий городов можно использовать `gram('Geox')`, однако нужно учитывать, что в названиях городов могут встречаться `прилагательные` (Нижний/Великий Новгород), предлог `на` (Комсомольск-на-Амуре) или `-` (Санкт-Петербург). Чтобы нам было чуть проще, можно учитывать то, что в большинстве случаев одно из слов в названиях городов из нескольких слов является топонимом.

In [22]:
before_city = morph_pipeline(['город', 'гор.', 'гор', 'г.', 'г'])

In [91]:
CITY = or_(
    rule(before_city.optional(), or_(
        rule(gram('ADJF').optional(), gram('Geox')), #Нижний Новгород
        rule(gram('NOUN'), HYPHEN, gram('Geox')), #Санкт-Петербург
        rule(gram('Geox'), HYPHEN, gram('NOUN')), #Пыть-Ях
        rule(gram('Geox'), HYPHEN, gram('PREP'), HYPHEN, gram('Geox')))), #Комсомольск-на-Амуре
    rule(before_city, gram('ADJF'))).interpretation(Address.city) #город Видное

#### Улица и дом

Чаще всего название улицы представляет собой сочетание слова `улица (ул.)` и слов(а) в родительном падеже:
* улица Невзоровых
* улица Героя Усилова
* улица 30 лет Победы

Маркером того, что перед нами именно название улицы, а не просто слово в родительном падеже, может служить следующий за улицей номер дома:
* улица Невзоровых, 64/2
* Невзоровых, 64/2

Либо специальное слово:
* проезд Мунарева
* проспект Комсомольский

Либо все вместе:
* проспект Комсомольский, 12

In [61]:
before_street = morph_pipeline(['улица', 'ул', 'ул.', 'тракт', 'проезд', 'вал', 'проспект', 'микрорайон', 
                                'аллеи', 'шоссе', 'маршала', 'героя', 'генерала армии', 'бульвар'])

Условно все улицы в текстах можно поделить на 2 группы:
* С номером дома
* Без номера дома

Понятное дело, что улицу с указанным номером дома будет легче распознать. Если же номера дома нет, нужно ориентироваться на вспомогательные слова (улица, проезд, проспект и пр.)

In [189]:
STREET_WITHOUT_HOUSE = or_(
    rule(before_street, NUMBER, or_(gram('NOUN'), gram('ADJF')), gram('NOUN')), #улица 70 лет Октября
    rule(before_street.optional(), before_street, gram('gent'), gram('gent').optional()), #улица героя Усилова/героя Усилова
    rule(before_street, gram('ADJF')), #проспект комсомольский
    rule(gram('ADJF'), before_street), #тюменский тракт
    rule(NUMBER, and_(not_(gram('Apro')), gram('ADJF'), gram('femn'))),
    ).interpretation(Address.street)

In [213]:
STREET_WITH_HOUSE = or_(
    rule(before_street.optional(), and_(gram('Name'), gram('gent')).optional(), or_(gram('Name'), gram('Surn'))), #Усилова
    rule(and_(gram('Name'), gram('gent')).optional(), gram('Surn')),
    rule(NUMBER.optional(), and_(not_(gram('Apro')), gram('ADJF')), gram('NOUN').optional()), #Октябрьская, 3 парковая, Старый Гай
    rule(and_(gram('gent'), not_(gram('NPRO')))), #артема
    ).interpretation(Address.street)

Перед номером дома могут стоять следующие слова:

In [35]:
before_building = morph_pipeline(['дом', 'дом номер', 'д.', 'д'])

Однако это необязательно, так как иногда номер дома пишут сразу после названия улицы.

In [36]:
HOUSE = rule(before_building.optional(), NUMBER, and_(not_(gram('VERB')), length_eq(1)).optional()
            ).interpretation(Address.building) #дом 12, дом 12а, 12а

В тестах есть ситуации, когда слово `дом` повторяется несколько раз. Учтем это:

In [37]:
HOUSE_REPEAT = rule(before_building.optional(), before_building.optional()) 

#### Корпус или строение

In [44]:
before_corpus = morph_pipeline(['корпус', 'корп', 'к', 'корп.', 'к.', 'строение', 'ст', '/'])

In [45]:
CORPUS = rule(before_corpus.optional(), NUMBER).interpretation(Address.corpus)

In [ ]:
CORPUS_NUMBER = rule(NUMBER).interpretation(Address.corpus)

#### Квартира

In [41]:
before_apartment = morph_pipeline(['квартира', 'кв.', 'кв'])

In [43]:
FLAT = rule(before_apartment.optional(), NUMBER).interpretation(Address.apartment)

Теперь попробуем составить правила:

In [386]:
ADDRESS = or_(
    rule(CITY), #город Москва
    rule(CITY.optional(), SEMICOLON.optional(), gram('gent').optional(), gram('gent').optional(), NUMBER.optional(), STREET_WITH_HOUSE, HOUSE_REPEAT.optional(), HOUSE, CORPUS.optional(), FLAT.optional()),
    rule(CITY.optional(), STREET_WITH_HOUSE, HOUSE_REPEAT.optional(), HOUSE, STREET_WITH_HOUSE, HOUSE, CITY.optional()),
    rule(CITY.optional(), HOUSE.optional(), STREET_WITHOUT_HOUSE, CITY.optional(), HOUSE_REPEAT.optional(), HOUSE.optional()),
    rule(CITY.optional(), STREET_WITHOUT_HOUSE, HOUSE_REPEAT.optional(), HOUSE, CORPUS.optional(), FLAT.optional()),
    rule(CITY.optional(), NUMBER, STREET_WITH_HOUSE, HOUSE_REPEAT.optional(), HOUSE, CORPUS.optional()),
    ).interpretation(Address)

In [387]:
ADDRESS_PARSER = Parser(ADDRESS)

#### Тесты для пункта 2 (8+16+11+6+30+32=103 теста)

In [388]:
class TestAddress(unittest.TestCase):
    
    def setUp(self):
        self.nothing = Address(city = None, street = None, building = None, corpus = None, apartment = None)
        self.parser = ADDRESS_PARSER
        
    def getRes(self, testing_address):
        find = self.parser.find(testing_address)
        res = self.nothing if find is None else find.fact
        return res
    
    #Квартиры
    def test_1(self):
        res = self.getRes('проспект комсомольский 50')
        self.assertEqual(res.apartment, None)

    def test_2(self):
        res = self.getRes('город липецк улица катукова 36 a')
        self.assertEqual(res.apartment, None)

    def test_3(self):
        res = self.getRes('сургут улица рабочая дом 31а')
        self.assertEqual(res.apartment, None)

    def test_4(self):
        res = self.getRes('город липецк доватора 18')
        self.assertEqual(res.apartment, None)

    def test_5(self):
        res = self.getRes('ну бехтеева 9 квартира 310')
        self.assertEqual(res.apartment, 'квартира 310')
        
    def test_6(self):
        res = self.getRes('Кусковская 19 корпус 1')
        self.assertEqual(res.apartment, None)

    def test_7(self):
        res = self.getRes('марша захарова 12 маршала захарова дом 12')
        self.assertEqual(res.apartment, None)

    def test_8(self):
        res = self.getRes('null')
        self.assertEqual(res.apartment, None)
    
    #Города
    def test_9(self):
        res = self.getRes('проспект комсомольский 50')
        self.assertEqual(res.city, None)

    def test_10(self):
        res = self.getRes('город липецк улица катукова 36 a')
        self.assertEqual(res.city, 'город липецк')

    def test_11(self):
        res = self.getRes('сургут улица рабочая дом 31а')
        self.assertEqual(res.city, 'сургут')

    def test_12(self):
        res = self.getRes('город липецк доватора 18')
        self.assertEqual(res.city, 'город липецк')

    def test_13(self):
        res = self.getRes('ну бехтеева 9 квартира 310')
        self.assertEqual(res.city, None)

    def test_14(self):
        res = self.getRes('сургут югорская 30/2')
        self.assertEqual(res.city, 'сургут')

    def test_15(self):
        res = self.getRes('индекс 12 мне вот этого не надо')
        self.assertEqual(res.city, None)

    def test_16(self):
        res = self.getRes('ты сургут улица 30 лет победы')
        self.assertEqual(res.city, 'сургут')

    def test_17(self):
        res = self.getRes('надо 50% город нальчик горького 1257')
        self.assertEqual(res.city, 'город нальчик')

    def test_18(self):
        res = self.getRes('null')
        self.assertEqual(res.city, None)

    def test_19(self):
        res = self.getRes('60 мегабит я')
        self.assertEqual(res.city, None)

    def test_20(self):
        res = self.getRes('сургут крылова 53/4')
        self.assertEqual(res.city, 'сургут')

    def test_21(self):
        res = self.getRes('так москва хамовнический вал но я думаю что я еще обсужу со своими домашними то есть вот у нас цифровое телевидение есть но акадо вот вы не спешите я тогда вам наберу но либо в приложения')
        self.assertEqual(res.city, 'москва')

    def test_22(self):
        res = self.getRes('мое 3 парковая')
        self.assertEqual(res.city, None)

    def test_23(self):
        res = self.getRes('Пришвина 17')
        self.assertEqual(res.city, None)

    def test_24(self):
        res = self.getRes('Старый Гай 1 корпус 2')
        self.assertEqual(res.city, None)
    
    #Номера домов и корпусов
    def test_25(self):
        res = self.getRes('проспект комсомольский 50')
        self.assertEqual(res.building, '50')

    def test_26(self):
        res = self.getRes('город липецк улица катукова 36 a')
        self.assertEqual(res.building, '36 a')

    def test_27(self):
        res = self.getRes('сургут улица рабочая дом 31а')
        self.assertEqual(res.building, 'дом 31а')

    def test_28(self):
        res = self.getRes('город липецк доватора 18')
        self.assertEqual(res.building, '18')

    def test_29(self):
        res = self.getRes('ну бехтеева 9 квартира 310')
        self.assertEqual(res.building, '9')

    def test_30(self):
        res = self.getRes('артема 32 квартира 8')
        self.assertEqual(res.building, '32')

    def test_31(self):
        res = self.getRes('город липецк полиграфическая дом 4')
        self.assertEqual(res.building, 'дом 4')

    def test_32(self):
        res = self.getRes('сколько стоит нет arkadata у нас есть москва каширское шоссе 55 корпус 1')
        self.assertEqual(res.building, '55')
        self.assertEqual(res.corpus, 'корпус 1')

    def test_33(self):
        res = self.getRes('люберцы октябрьский проспект 10 корпус 1')
        self.assertEqual(res.building, '10')
        self.assertEqual(res.corpus, 'корпус 1')

    def test_34(self):
        res = self.getRes('бульвар миттова 24')
        self.assertEqual(res.building, '24')

    def test_35(self):
        res = self.getRes('стол вы знаете москва алтуфьевское 78')
        self.assertEqual(res.building, '78')
        
    #Адрес Санкт-Петербурга
    def test_36(self):
        res = self.getRes('санкт-петербург школьная 20')
        self.assertEqual('санкт-петербург', res.city)
        self.assertEqual('школьная', res.street)
        self.assertEqual('20', res.building)

    def test_37(self):
        res = self.getRes('санкт-петербург юрия гагарина 22 к2')
        self.assertEqual('санкт-петербург', res.city)
        self.assertEqual('юрия гагарина', res.street)
        self.assertEqual('22', res.building)
        self.assertEqual('к2', res.corpus)

    def test_38(self):
        res = self.getRes('питер гагарина 22 к2')
        self.assertEqual('питер', res.city)
        self.assertEqual('гагарина', res.street)
        self.assertEqual('22', res.building)
        self.assertEqual('к2', res.corpus)

    def test_39(self):
        res = self.getRes("санкт-петербург;юнтоловский 43 корпус 1")
        self.assertEqual('санкт-петербург', res.city)
        self.assertEqual('юнтоловский', res.street)
        self.assertEqual('43', res.building)
        self.assertEqual('корпус 1', res.corpus)

    def test_40(self):
        res = self.getRes("санкт-петербург;юнтоловский 43 строение 1")
        self.assertEqual('санкт-петербург', res.city)
        self.assertEqual('юнтоловский', res.street)
        self.assertEqual('43', res.building)
        self.assertEqual('строение 1', res.corpus)

    def test_41(self):
        res = self.getRes("юнтоловский 43 строение 1")
        self.assertEqual('юнтоловский', res.street)
        self.assertEqual('43', res.building)
        self.assertEqual('строение 1', res.corpus)
        
    #Полные адреса
    def test_42(self):
        res = self.getRes('проспект комсомольский 50')
        self.assertEqual( 'проспект комсомольский', res.street)
        self.assertEqual('50', res.building)

    def test_43(self):
        res = self.getRes('город липецк улица катукова 36 a')
        self.assertEqual('город липецк', res.city)
        self.assertEqual('улица катукова', res.street)
        self.assertEqual('36 a', res.building)

    def test_44(self):
        res = self.getRes('сургут улица рабочая дом 31а')
        self.assertEqual('сургут', res.city)
        self.assertEqual('улица рабочая', res.street)
        self.assertEqual('дом 31а', res.building)

    def test_45(self):
        res = self.getRes('город липецк доватора 18')
        self.assertEqual('город липецк', res.city)
        self.assertEqual('доватора', res.street)
        self.assertEqual('18', res.building)

    def test_46(self):
        res = self.getRes('ну бехтеева 9 квартира 310')
        self.assertEqual(None, res.city)
        self.assertEqual('бехтеева', res.street)
        self.assertEqual('9', res.building)
        self.assertEqual('квартира 310', res.apartment)

    def test_47(self):
        res = self.getRes('московская 34б')
        self.assertEqual(None, res.city)
        self.assertEqual( 'московская', res.street)
        self.assertEqual('34б', res.building)
        self.assertEqual(None, res.apartment)

    def test_48(self):
        res = self.getRes('улица минина дом 1')
        self.assertEqual(None, res.city)
        self.assertEqual('улица минина', res.street)
        self.assertEqual('дом 1', res.building)
        self.assertEqual( None, res.apartment)

    def test_49(self):
        res = self.getRes('сколько улица 30 лет победы 50/46')
        self.assertEqual(None, res.city)
        self.assertEqual('улица 30 лет победы', res.street)
        self.assertEqual('50', res.building)
        self.assertEqual('/46', res.corpus)

    def test_50(self):
        res = self.getRes('тюменский тракт 10')
        self.assertEqual(None, res.city, )
        self.assertEqual('тюменский тракт', res.street)
        self.assertEqual('10', res.building)
        self.assertEqual(None, res.apartment)

    def test_51(self):
        res = self.getRes('береговая 43 береговая 43 сургут')
        self.assertEqual('сургут', res.city)
        self.assertEqual('береговая', res.street)
        self.assertEqual('43', res.building)
        self.assertEqual(None, res.apartment)

    def test_52(self):
        res = self.getRes('сургут югорская 30/2')
        self.assertEqual('сургут', res.city)
        self.assertEqual('югорская', res.street)
        self.assertEqual('30', res.building)
        self.assertEqual('/2', res.corpus)
        self.assertEqual( None, res.apartment)

    def test_53(self):
        res = self.getRes('индекс 12 мне вот этого не надо')
        self.assertEqual(None, res.city, )
        self.assertEqual(None, res.street,)
        self.assertEqual(None, res.building)
        self.assertEqual(None, res.apartment)

    def test_54(self):
        res = self.getRes('город сургут улица фармана салманова 4')
        self.assertEqual('город сургут', res.city)
        self.assertEqual('улица фармана салманова', res.street)
        self.assertEqual('4', res.building)
        self.assertEqual( None, res.apartment)

    def test_55(self):
        res = self.getRes('зеленые аллеи город видное дом 8')
        self.assertEqual('город видное', res.city, )
        self.assertEqual('зеленые аллеи', res.street)
        self.assertEqual('дом 8', res.building)
        self.assertEqual( None, res.apartment)

    def test_56(self):
        res = self.getRes('зелинского улица зелинского дом 2')
        self.assertEqual(None, res.city,)
        self.assertEqual('улица зелинского', res.street)
        self.assertEqual('дом 2', res.building)
        self.assertEqual(None, res.apartment)

    def test_57(self):
        res = self.getRes('Кусковская 19 корпус 1 ')
        self.assertEqual(None, res.city)
        self.assertEqual('Кусковская', res.street)
        self.assertEqual('19', res.building)
        self.assertEqual('корпус 1', res.corpus)
        self.assertEqual(None, res.apartment)

    def test_58(self):
        res = self.getRes('москва щелковское шоссе 35')
        self.assertEqual('москва', res.city)
        self.assertEqual('щелковское шоссе', res.street)
        self.assertEqual('35', res.building)
        self.assertEqual(None, res.apartment)

    def test_59(self):
        res = self.getRes('город москва марьинский парк дом 25 корпус 2')
        self.assertEqual('город москва', res.city,)
        self.assertEqual('марьинский парк', res.street,)
        self.assertEqual('дом 25', res.building)
        self.assertEqual('корпус 2', res.corpus)
        self.assertEqual(None, res.apartment)

    def test_60(self):
        res = self.getRes('Старый Гай 1 корпус 2')
        self.assertEqual('Старый Гай'.lower(), res.street.lower(), None)
        self.assertEqual('1', res.building)
        self.assertEqual('корпус 2', res.corpus)
        self.assertEqual(None, res.apartment)

    def test_61(self):
        res = self.getRes('улица 3 почтовое отделение дом 62')
        self.assertEqual(None, res.city)
        self.assertEqual('улица 3 почтовое отделение', res.street)
        self.assertEqual('дом 62', res.building)
        self.assertEqual(None, res.apartment)

    def test_62(self):
        res = self.getRes('нижний новгород улица июльских дней 19')
        self.assertEqual('нижний новгород', res.city,)
        self.assertEqual('улица июльских дней', res.street)
        self.assertEqual('19', res.building)
        self.assertEqual(None, res.apartment)

    def test_63(self):
        res = self.getRes('так москва хамовнический вал но я думаю что я еще обсужу со своими домашними то есть вот у нас цифровое телевидение есть но акадо вот вы не спешите я тогда вам наберу но либо в приложения')
        self.assertEqual('москва', res.city)
        self.assertEqual('хамовнический вал', res.street)
        self.assertEqual(None, res.building)
        self.assertEqual(None, res.apartment)

    def test_64(self):
        res = self.getRes('город сургут улица семена билецкого 1')
        self.assertEqual('город сургут', res.city)
        self.assertEqual('улица семена билецкого', res.street)
        self.assertEqual('1', res.building)
        self.assertEqual(None, res.apartment)
    
    def test_65(self):
        res = self.getRes('улица значит антонова овсиенко дом 19/2')
        self.assertEqual('антонова овсиенко', res.street)
        self.assertEqual('дом 19', res.building)
        self.assertEqual('/2', res.corpus)

    def test_66(self):
        res = self.getRes('улица генерала армии епишева дом 9')
        self.assertEqual('улица генерала армии епишева', res.street)
        self.assertEqual('дом 9', res.building)

    def test_67(self):
        res = self.getRes('улица академика байкова дом 9')
        self.assertEqual('дом 9', res.building)
        self.assertEqual('улица академика байкова', res.street)

    def test_68(self):
        res = self.getRes('улица академика байкова дом дом дом 9')
        self.assertEqual('дом 9', res.building)
        self.assertEqual('улица академика байкова', res.street)

    def test_69(self):
        res = self.getRes('улица подзаборного байкова дом дом дом 9')
        self.assertEqual('дом 9', res.building)
        self.assertEqual('улица подзаборного байкова', res.street)

    def test_70(self):
        res = self.getRes('улица монтажника байкова дом дом дом 9')
        self.assertEqual('дом 9', res.building)
        self.assertEqual('улица монтажника байкова', res.street)

    def test_71(self):
        res = self.getRes('такзначит у меня дом номер 12 а улица джона рида')
        self.assertEqual('улица джона рида', res.street)
        self.assertEqual('дом номер 12 а', res.building)
        
    #Улицы
    def test_72(self):
        res = self.getRes('проспект комсомольский 50')
        self.assertEqual('проспект комсомольский', res.street)

    def test_73(self):
        res = self.getRes('город липецк улица катукова 36 a')
        self.assertEqual('улица катукова', res.street)

    def test_74(self):
        res = self.getRes('сургут улица рабочая дом 31а')
        self.assertEqual('улица рабочая', res.street)

    def test_75(self):
        res = self.getRes('город липецк доватора 18')
        self.assertEqual('доватора', res.street)

    def test_76(self):
        res = self.getRes('ну бехтеева 9 квартира 310')
        self.assertEqual('бехтеева', res.street)

    def test_77(self):
        res = self.getRes('улица меркулова дом 24')
        self.assertEqual('улица меркулова', res.street)

    def test_78(self):
        res = self.getRes('октябрьская 48 частный дом город сургут')
        self.assertEqual('октябрьская', res.street)

    def test_79(self):
        res = self.getRes('сколько улица 30 лет победы 50 46')
        self.assertEqual('улица 30 лет победы', res.street)

    def test_80(self):
        res = self.getRes('тюменский тракт 10')
        self.assertEqual('тюменский тракт', res.street)

    def test_81(self):
        res = self.getRes('сургут югорская 30/2')
        self.assertEqual('югорская', res.street)

    def test_82(self):
        res = self.getRes('индекс 12 мне вот этого не надо')
        self.assertEqual(None, res.street)

    def test_83(self):
        res = self.getRes('старый оскол микрорайон олимпийский дом 23 квартира 105')
        self.assertEqual('микрорайон олимпийский', res.street)

    def test_84(self):
        res = self.getRes('город сургут улица фармана салманова 4')
        self.assertEqual('улица фармана салманова', res.street)

    def test_85(self):
        res = self.getRes('ты сургут улица 30 лет победы')
        self.assertEqual('улица 30 лет победы', res.street)

    def test_86(self):
        res = self.getRes('проезд мунарева 2')
        self.assertEqual('проезд мунарева', res.street)

    def test_87(self):
        res = self.getRes('домашний адрес где я живу')
        self.assertEqual(None, res.street)

    def test_88(self):
        res = self.getRes('артема 32 квартира 8')
        self.assertEqual('артема', res.street)

    def test_89(self):
        res = self.getRes('знаете знаете у меня дорогая девочка у меня уже все есть и менять из из одного переходить на другой я не хочу поэтому какой город квартира какой ничего я вам сообщать не хочу поэтому до свидания я ничего не')
        self.assertEqual(None, res.street)

    def test_90(self):
        res = self.getRes('новогиреевская 34')
        self.assertEqual('новогиреевская', res.street)

    def test_91(self):
        res = self.getRes('мое 3 парковая')
        self.assertEqual('3 парковая', res.street)

    def test_92(self):
        res = self.getRes('москва мусы джалиля 38 корпус 2')
        self.assertEqual('мусы джалиля', res.street)

    def test_93(self):
        res = self.getRes('надо 50% город нальчик горького 1257')
        self.assertEqual('горького', res.street)

    def test_94(self):
        res = self.getRes('сколько стоит нет arkadata у нас есть москва каширское шоссе 55 корпус 1')
        self.assertEqual('каширское шоссе', res.street)

    def test_95(self):
        res = self.getRes('зеленые аллеи город видное дом 8')
        self.assertEqual('зеленые аллеи', res.street)

    def test_96(self):
        res = self.getRes('дмитрия ульянова 17 корпус 1 москва')
        self.assertEqual('дмитрия ульянова', res.street)

    def test_97(self):
        res = self.getRes('null')
        self.assertEqual(None, res.street)

    def test_98(self):
        res = self.getRes('стол вы знаете москва алтуфьевское 78')
        self.assertEqual('алтуфьевское', res.street)

    def test_99(self):
        res = self.getRes('марша захарова 12 маршала захарова дом 12')
        self.assertEqual('маршала захарова', res.street)

    def test_100(self):
        res = self.getRes('а зачем')
        self.assertEqual(None, res.street)

    def test_101(self):
        res = self.getRes('Кавказский 16')
        self.assertEqual('Кавказский', res.street)

    def test_102(self):
        res = self.getRes('Старый Гай 1 корпус 2')
        self.assertEqual('Старый Гай', res.street)

    def test_103(self):
        res = self.getRes('зелинского улица зелинского дом 2')
        self.assertEqual('улица зелинского', res.street)
    

In [389]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..................................................................................................................
----------------------------------------------------------------------
Ran 114 tests in 2.913s

OK
